<a href="https://colab.research.google.com/github/adity1234567/Covid-HeRA/blob/main/Drink_Bleach_or_Do_What_Now%3F_Covid_HeRA_A_Study_of_Risk_Informed_Health_Decision_Making_in_the_Presence_of_COVID_19_Misinformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install tweepy pandas numpy
#tweepy is needed for twitter API

# Clone the CoAID Dataset

In [ ]:
!git clone https://github.com/cuilimeng/CoAID.git


Cloning into 'CoAID'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 83 (delta 23), reused 46 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 3.97 MiB | 12.01 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
%cd CoAID/05-01-2020/

/content/CoAID/05-01-2020


In [ ]:
!ls

ClaimFakeCOVID-19.csv		      NewsFakeCOVID-19.csv
ClaimFakeCOVID-19_tweets.csv	      NewsFakeCOVID-19_tweets.csv
ClaimFakeCOVID-19_tweets_replies.csv  NewsFakeCOVID-19_tweets_replies.csv
ClaimRealCOVID-19.csv		      NewsRealCOVID-19.csv
ClaimRealCOVID-19_tweets.csv	      NewsRealCOVID-19_tweets.csv
ClaimRealCOVID-19_tweets_replies.csv  NewsRealCOVID-19_tweets_replies.csv


# Upload Twitter API Credentials

In [ ]:
api_key = "OAUTH_CONSUMER_KEY"
api_secret = "OAUTH_CONSUMER_SECRET"

with open("twitter_key.txt", "w") as f:
    f.write(f"{api_key},{api_secret}")


In [ ]:
# Read and print the content of the file to verify
with open('twitter_key.txt', 'r') as f:
    print(f.read())


OAUTH_CONSUMER_KEY,OAUTH_CONSUMER_SECRET


In [ ]:
import os

# List contents of the directory to verify if the file exists
os.listdir('/content/CoAID/05-01-2020/')


['NewsFakeCOVID-19_tweets_replies.csv',
 'twitter_key.txt',
 'ClaimFakeCOVID-19_tweets_replies.csv',
 'ClaimRealCOVID-19_tweets.csv',
 'ClaimRealCOVID-19.csv',
 'ClaimFakeCOVID-19_tweets.csv',
 'ClaimFakeCOVID-19.csv',
 'ClaimRealCOVID-19_tweets_replies.csv',
 'NewsFakeCOVID-19_tweets.csv',
 'NewsRealCOVID-19_tweets_replies.csv',
 'NewsRealCOVID-19_tweets.csv',
 'NewsFakeCOVID-19.csv',
 'NewsRealCOVID-19.csv']

# extarct the data

In [ ]:
from google.colab import files

#uploaded=files.upload()

1. os is used to join paths and create directories or handle file paths dynamically in the script.
This ensures that paths are platform-independent (works across Windows, macOS, Linux).

2. time can be used for delays, at the time of dealing with rate limits or API restrictions

3. import argparse
  
  **Purpose:** argparse is a Python library that makes it easy to write user-friendly command-line interfaces. It provides functionality for parsing command-line arguments passed to the script.
  
  **Why it's used**: In your script, argparse is used to accept the argument for data_type (either real or fake news). This allows you to customize how you run the script directly from the command line or modify it in Colab as an input argument.

In [ ]:
import os
import time
import pandas as pd
import tweepy
import argparse #enables you to pass parameters to your script via the command line.
from joblib import Parallel,delayed

*Change the folder path as needed*

In [ ]:
folder_path="/content/CoAID/05-01-2020/"

**Get tweets id's from csv**

In [ ]:
def get_ids_from_csv(filename,id_columns):
  df=pd.read_csv(os.path.join(folder_path,filename))
  tweet_ids= df[id_columns].values.flatten()
  return list(map(str,tweet_ids))


**Fetch Fake Tweet IDs**

In [ ]:
def get_fake_tweet_ids():
    tweet_ids = set()
    fake_tweet_paths = ["ClaimFakeCOVID-19_tweets.csv", "ClaimFakeCOVID-19_tweets_replies.csv"]
    tweet_ids = tweet_ids.union(set(get_ids_from_csv(fake_tweet_paths[0], ["tweet_id"])))
    tweet_ids = tweet_ids.union(set(get_ids_from_csv(fake_tweet_paths[1], ["tweet_id", "reply_id"])))

    real_tweet_paths = ["NewsFakeCOVID-19_tweets.csv", "NewsFakeCOVID-19_tweets_replies.csv"]
    tweet_ids = tweet_ids.union(set(get_ids_from_csv(real_tweet_paths[0], ["tweet_id"])))
    tweet_ids = tweet_ids.union(set(get_ids_from_csv(real_tweet_paths[1], ["tweet_id", "reply_id"])))
    return list(tweet_ids)

Fetch Real Tweet IDs

In [ ]:
def get_real_tweet_ids():
    tweet_ids = set()
    real_tweet_paths = ["ClaimRealCOVID-19_tweets.csv", "ClaimRealCOVID-19_tweets_replies.csv"]
    tweet_ids = tweet_ids.union(set(get_ids_from_csv(real_tweet_paths[0], ["tweet_id"])))
    tweet_ids = tweet_ids.union(set(get_ids_from_csv(real_tweet_paths[1], ["tweet_id", "reply_id"])))

    news_paths = ["NewsRealCOVID-19_tweets.csv", "NewsRealCOVID-19_tweets_replies.csv"]
    tweet_ids = tweet_ids.union(set(get_ids_from_csv(news_paths[0], ["tweet_id"])))
    tweet_ids = tweet_ids.union(set(get_ids_from_csv(news_paths[1], ["tweet_id", "reply_id"])))
    return list(tweet_ids)


**Fetch tweets from Twitter API**

In [ ]:
def get_status(api, tweet_id):
    try:
        tweet = api.get_status(tweet_id, tweet_mode="extended")
        return True, {"id": tweet.id, "raw_text": tweet.full_text}
    except tweepy.TweepError:
        return False, None

In [ ]:
def main(label_type: int = 1):
    if label_type == 1:
        tweet_ids = get_real_tweet_ids()
        new_csv_path = os.path.join("coaid_tweets_real.csv")

        #new_csv_path = "/content/drive/MyDrive/coaid_tweets_real.csv"
    elif label_type == 2:
        tweet_ids = get_fake_tweet_ids()
        new_csv_path = os.path.join("coaid_tweets_fake.csv")
        #new_csv_path = os.path.join(folder_path, "coaid_tweets_fake.csv")
       # new_csv_path = "/content/drive/MyDrive/coaid_tweets_fake.csv"
    else:
        print("Label Type unspecified. Quitting...")
        return None

    # Authenticate Twitter API
    api_objs = []
    for key, secret in consumer_credentials:
        auth = tweepy.OAuthHandler(key, secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)
        api_objs.append(api)

    df = pd.DataFrame(columns=["id", "raw_text"])
    start_time = time.time()

    try:
        print("Start grabbing tweets")
        chunk_size = 100 * len(api_objs)
        number_of_parallel_jobs = 32 * len(api_objs)
        backend = "threading"

        for index in range(0, len(tweet_ids), chunk_size):
            start = index
            end = min(index + chunk_size, len(tweet_ids))

            parallel_tweet_ids = tweet_ids[start:end]
            parallel_api_args = [(api_objs[i % len(api_objs)], parallel_tweet_ids[i]) for i in range(len(parallel_tweet_ids))]

            results = Parallel(n_jobs=number_of_parallel_jobs, backend=backend)(
                delayed(get_status)(*tweet_arg) for tweet_arg in parallel_api_args
            )

            for save_tweet, row in results:
                if save_tweet:
                    df = df.append(row, ignore_index=True)

            if index % 100 == 0:
                print(f"Progress: {index}")

    except Exception as e:
        print(f"Encountered exception {e}. Saving progress...")
        df.to_csv(new_csv_path, index=False)
        return

    df.to_csv(new_csv_path, index=False)
    end_time = time.time()
    print(f"Total time taken: {end_time - start_time} seconds")


In [ ]:
consumer_credentials = [("OAUTH_CONSUMER_KEY", "OAUTH_CONSUMER_SECRET")] # Change this line to be list of tuples.

In [ ]:
data_type = 2  # Set to 1 for Real, 2 for Fake
main(data_type)

Start grabbing tweets
Encountered exception module 'tweepy' has no attribute 'TweepError'. Saving progress...


In [ ]:
def save_tweet_ids(label_type: int = 1):
    """Save tweet IDs as a CSV file in Google Drive."""
    if label_type == 1:
        tweet_ids = get_real_tweet_ids()
        csv_path = os.path.join(folder_path, "coaid_tweets_real.csv")
    elif label_type == 2:
        tweet_ids = get_fake_tweet_ids()
        csv_path = os.path.join(folder_path, "coaid_tweets_fake.csv")
    else:
        print("Label Type unspecified. Quitting...")
        return

    df = pd.DataFrame(tweet_ids, columns=["tweet_id"])
    df.to_csv(csv_path, index=False)
    print(f"CSV file saved at: {csv_path}")


# Call function to save CSV files
save_tweet_ids(label_type=1)  # Save real tweet IDs
save_tweet_ids(label_type=2)  # Save fake tweet IDs


CSV file saved at: /content/CoAID/05-01-2020/coaid_tweets_real.csv
CSV file saved at: /content/CoAID/05-01-2020/coaid_tweets_fake.csv


In [ ]:
real=pd.read_csv("/content/CoAID/07-01-2020/coaid_tweets_real.csv")
real

,tweet_id
0,1274809340738887686
1,1258507329873428480
2,1271827020197040128
3,1278032543632195584
4,1277683000822595584
...,...
106946,1278346784373387265
106947,1276652381338468352
106948,1274425345874309120
106949,1274808530290249736


In [ ]:
fake=pd.read_csv("/content/CoAID/07-01-2020/coaid_tweets_fake.csv")
fake

,tweet_id
0,1263163759163346944
1,1263313875874861056
2,1257716886713720833
3,1257323349719441409
4,1262239079044993024
...,...
2882,1260622967253409792
2883,1262151609498378240
2884,1256240063131848705
2885,1262365194501709826


# Merging all the datasets

In [ ]:

def merge_all_datasets_from_folders(folder_paths, output_filename):
    dataframes = []

    for folder in folder_paths:
        for filename in os.listdir(folder):
            if filename.endswith(".csv"):
                file_path = os.path.join(folder, filename)
                df = pd.read_csv(file_path)
                dataframes.append(df)

    merged_df = pd.concat(dataframes, ignore_index=True)


    merged_df.to_csv(output_filename, index=False)
    print(f"Merged dataset saved to {output_filename}")

In [ ]:
def merge_datasets_from_folders(folder_paths, output_filename):
    # Initialize an empty DataFrame to hold the merged data
    merged_data = pd.DataFrame()

    # Loop through the folder paths and read the CSV files
    for folder_path in folder_paths:
        if os.path.exists(folder_path):  # Check if the file exists
            # Read the CSV file into a DataFrame
            data = pd.read_csv(folder_path)
            merged_data = pd.concat([merged_data, data], ignore_index=True)  # Merge datasets

        else:
            print(f"Warning: {folder_path} does not exist or is incorrect.")

    # Save the merged dataset to the output filename
    merged_data.to_csv(output_filename, index=False)
    print(f"Merged dataset saved at: {output_filename}")

In [ ]:
folder_paths = ["/content/CoAID/05-01-2020/coaid_tweets_fake.csv", "/content/CoAID/07-01-2020/coaid_tweets_fake.csv"]

output_dir = "/content/drive/MyDrive/CoAID"
os.makedirs(output_dir, exist_ok=True)  # true use korar reason: eta jeno exist korle r new kore create na kore
output_filename = os.path.join(output_dir, "merged_fake_dataset.csv")
merge_datasets_from_folders(folder_paths, output_filename)

Merged dataset saved at: /content/drive/MyDrive/CoAID/merged_fake_dataset.csv


In [ ]:
folder_paths = ["/content/CoAID/11-01-2020/coaid_tweets_real.csv", "/content/CoAID/09-01-2020/coaid_tweets_real.csv","/content/CoAID/07-01-2020/coaid_tweets_real.csv","/content/CoAID/05-01-2020/coaid_tweets_real.csv"]

output_dir = "/content/drive/MyDrive/CoAID"
os.makedirs(output_dir, exist_ok=True)  # true use korar reason: eta jeno exist korle r new kore create na kore
output_filename = os.path.join(output_dir, "merged_real_dataset.csv")
merge_datasets_from_folders(folder_paths, output_filename)

Merged dataset saved at: /content/drive/MyDrive/CoAID/merged_real_dataset.csv


In [ ]:
merge_fake_dataset=pd.read_csv("/content/drive/MyDrive/CoAID/merged_fake_dataset.csv")
merge_fake_dataset

,tweet_id
0,1242844056285097984
1,1221328326113026048
2,1238360177059024896
3,1234204584068993032
4,1250902763585896448
...,...
18733,1260622967253409792
18734,1262151609498378240
18735,1256240063131848705
18736,1262365194501709826


In [ ]:
merge_real_dataset=pd.read_csv("/content/drive/MyDrive/CoAID/merged_fake_dataset.csv")
merge_real_dataset

,tweet_id
0,1242844056285097984
1,1221328326113026048
2,1238360177059024896
3,1234204584068993032
4,1250902763585896448
...,...
18733,1260622967253409792
18734,1262151609498378240
18735,1256240063131848705
18736,1262365194501709826


In [ ]:
def merge_all(label_path, real_path, fake_path):
    label_id = pd.read_csv(label_path)
    coaid_tweets_real_df = pd.read_csv(real_path)
    coaid_tweets_fake_df = pd.read_csv(fake_path)

    df_real_labels = label_id.merge(coaid_tweets_real_df, left_on='id', right_on='tweet_id', how='inner')
    df_fake_labels = label_id.merge(coaid_tweets_fake_df, left_on='id', right_on='tweet_id', how='inner')


    df_full = pd.concat([df_fake_labels, df_real_labels], ignore_index=True)
    df_full.to_csv('/content/drive/MyDrive/CoAID/coaid_full_dataset.csv', index=False)
    print("Full dataset saved as 'coaid_full_dataset.csv'")


In [ ]:
# Set the file paths
label_path = "/content/drive/MyDrive/CoAID/full_dataset_label_ids.csv"  # Update with the correct path
real_path = "/content/drive/MyDrive/CoAID/merged_real_dataset.csv"  # Update with the correct path
fake_path = "/content/drive/MyDrive/CoAID/merged_fake_dataset.csv"  # Update with the correct path

# Call the merge_all function
merge_all(label_path, real_path, fake_path)

Full dataset saved as 'coaid_full_dataset.csv'


In [ ]:
merge_dataset=pd.read_csv('/content/drive/MyDrive/CoAID/coaid_full_dataset.csv')
merge_dataset.head(10)

,label,index,id,tweet_id
0,3,100001,1252630938770649089,1252630938770649089
1,4,100001,1243968198111789058,1243968198111789058
2,4,100001,1242474839966765056,1242474839966765056
3,4,100001,1242461115616866304,1242461115616866304
4,4,100001,1242400142335983619,1242400142335983619
5,4,100001,1238131052582928385,1238131052582928385
6,4,100002,1254164579459854336,1254164579459854336
7,4,100003,1256227600936177664,1256227600936177664
8,4,100003,1256215023619395584,1256215023619395584
9,4,100003,1256167023148765186,1256167023148765186


In [ ]:
merge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96054 entries, 0 to 96053
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   label     96054 non-null  int64
 1   index     96054 non-null  int64
 2   id        96054 non-null  int64
 3   tweet_id  96054 non-null  int64
dtypes: int64(4)
memory usage: 2.9 MB


In [ ]:
merge_dataset.describe()

,label,index,id,tweet_id
count,96054.000000,96054.000000,9.605400e+04,9.605400e+04
mean,0.137267,7609.208060,1.247078e+18,1.247078e+18
std,0.515197,24865.108326,9.110816e+15,9.110816e+15
min,0.000000,0.000000,1.200930e+18,1.200930e+18
25%,0.000000,561.000000,1.242296e+18,1.242296e+18
50%,0.000000,1168.000000,1.248694e+18,1.248694e+18
75%,0.000000,1325.000000,1.255197e+18,1.255197e+18
max,4.000000,100165.000000,1.256373e+18,1.256373e+18
